In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('./datasets/Data_unido_2018_2021.csv')
data.head(2)

C:\Users\rod_e\AppData\Local\Temp\ipykernel_8552\1256429594.py:1: DtypeWarning: Columns (13,28,29,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./datasets/Data_unido_2018_2021.csv')


,año,mes,fecha_ingreso,referida,estado_civil,edad,institucion_derechohab,nivel_edu,ocupacion,religion,...,semanas_gestacion_usg,dias_gestacion_usg,firma_consentimiento,se_complica,con_dolor,analgesico,anticonceptivo_post,fecha_cierre,resultado_ile,procedimiento_ile
0,2016,ABRIL,2016-04-01,NaN,SOLTERA,24,NINGUNO,PREPARATORIA,ESTUDIANTE,NINGUNA,...,3.0,NaN,NaN,NaN,NaN,NaN,CONDON,2016-04-08,COMPLETA,MEDICAMENTO
1,2016,ABRIL,2016-04-01,NaN,SOLTERA,30,NINGUNO,SECUNDARIA,TRABAJADORA DEL HOGAR NO REMUNERADA,CATOLICA,...,6.0,NaN,NaN,NaN,NaN,NaN,CONDON,2016-04-08,COMPLETA,MEDICAMENTO


In [31]:
def tabla_hechos():
    persona = pd.read_csv('./tablas/tabla_persona.csv')
    ile = pd.read_csv('./tablas/ILE.csv')
    tiempo = pd.read_csv('./tablas/tiempojunta.csv')
    geo = pd.read_csv('./tablas/Geografia.csv')
    ile = pd.read_csv('./tablas/ILE.csv')
    abor = pd.read_csv('./datasets/Data_unido_2018_2021.csv')

    hecho = {
        'Id_aborto' : ['ile' + str(i) for i in range(np.shape(persona)[0])],
        'Id_tiempo' : [(i) for i in tiempo['Id_tiempo'] if i.startswith('ile')],
        'Id_persona' : persona['Id'],
        'Id_entidad' : geo['Id_Geo'],
        'Id_ILE' : ile['id_ile'],
        'Semanas_gestacion' : abor['semanas_gestacion_usg'],
        'Dias_gestacion' : abor['dias_gestacion_usg']    
    }
    
    pd.DataFrame(hecho).to_csv('./tablas/Tabla_Ile_hechos.csv')

In [32]:
tabla_hechos()

C:\Users\rod_e\AppData\Local\Temp\ipykernel_8552\1702054185.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  tiempo = pd.read_csv('./tablas/tiempojunta.csv')
C:\Users\rod_e\AppData\Local\Temp\ipykernel_8552\1702054185.py:7: DtypeWarning: Columns (13,28,29,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  abor = pd.read_csv('./datasets/Data_unido_2018_2021.csv')


In [3]:
def generar_tabla_tiempo(x, path = './tablas/tabla_tiempo_ile.csv', columna = 'fecha_ingreso', ile = False):
    """
    Función para generar tabla de tiempo, en esta función toma una columna 
    y la divide en sus partes (dia, mes, año), crea un id para la tabla, 
    si es ILE agrega eso más la fecha junta.
    
    x : dataframe que proviene de un archivo csv
    path : ubicacion para guardar el archivo csv que genera esta función
    columna : Columna que seleccionamos del dataframe para separar dicha fecha 
    maternidad : valor booleano que define que tipo de dataset va a ser tratado
    """
    
    if ile:
        # Guardamos la columna en una varialbe temporal
        fechas = x[columna]
        
        # Creamos los valores de cada columna
        id = ['ile' + ''.join(fecha.strip()) for fecha in fechas]
        dia = [fecha.strip()[2] for fecha in fechas]
        mes = [fecha.strip()[1] for fecha in fechas]
        anio = [fecha.strip()[0] for fecha in fechas]
        ile = [1 for fecha in fechas]
        fecha_nac = [float('nan') for fecha in fechas]
        natalidad = [0 for fecha in fechas]
        fecha_ing = x['fecha_ingeso']
        
        # Creamos un diccionario que posteriormente sera un archivo csv
        tabla_tiempo_ile = {
            'Id_tiempo':id,
            'Año' : anio,
            'Mes': mes,
            'Dia' : dia,
            'Fecha_ingreso' : fecha_ing,
            'Fecha_nacimiento' : fecha_nac,
            'ILE' : ile,
            'Natalidad' : natalidad
        }
        
        # Guardamos el archivo csv
        pd.DataFrame(tabla_tiempo_ile).to_csv(path, index=False)
    
    else:
        fechas = x[columna]
        
        # Creamos las columnas del archivo csv
        anio = [fecha.strip()[0] for fecha in fechas]
        mes = [fecha.strip()[1] for fecha in fechas]
        dia = [fecha.strip()[2] for fecha in fechas]
        id = ['nat' + ''.join(fecha.strip()) for fecha in fechas]
        ile = [float('nan') for fecha in fechas]
        fecha_nac = [fecha for fecha in fechas]
        natalidad = [1 for fecha in fechas]
        fecha_ing = [float('nan') for fecha in fechas]
        
        # Creamos el diccionario que posteriormente sera nuestro archivo csv
        tabla_tiempo = {
            'Id_tiempo' : id,
            'Año' : anio,
            'Mes': mes,
            'Dia' : dia,
            'Fecha_ingreso' : fecha_ing,
            'Fecha_nacimiento' : fecha_nac,
            'ILE' : ile,
            'Natalidad' : natalidad            
        }
        
        # Intentamos abrir si ya existe el otro archivo
        try:
            uni = pd.read_csv(path)
            
            juntos = uni.append(pd.DataFrame(tabla_tiempo))
            juntos.to_csv('./tablas/tiempojunta.csv', index=False)
        except:
            print('No es posible escribir el archivo')

In [82]:
def medicina(x, path = './tablas/medicina.csv'):
    """
    Funciona para crear la tabla de datos de las personas, enofacado más en los parametros sexuales

    x : dataFrame
    path : ubicación donde guardar el archivo csv
    """
    dat = x[x.columns[13:27].values]

    pd.DataFrame(dat).to_csv(path, index=False)
    

In [123]:
def get_table_medicina(x = pd.DataFrame(), path ='./tablas/medicina.csv'):
    """
    Funcion que genera tabla de medicina para el archivo ILE del proyecto
    x : Data frame del archivo previamente leido
    path : ubicacion donde se va a guardar el archivo generado por esta función.
    """
    med = x[x.columns[14:28].values]
    id_med = [i for i in range(np.shape(med)[0])]
    
    med.insert(0, 'ID_med', id_med)

    pd.DataFrame(med).to_csv(path, index = False)

In [130]:
def get_table_ILE(x = pd.DataFrame(), path = './tablas/ILE.csv'):
    """
    Función que sirve para generar la tabla de ILE, que contiene datos especificos 
    acerca del ILE de cada persona.
    
    x : Data frame del archivo csv previamente leido
    path : ubicacion donde se va a guardar el archivo csv que genera esta función
    """

    mot = x['motivos_interrupcion']
    analg = x['analgesico']
    anti = x['anticonceptivo_post']
    res = x['resultado_ile']
    procd = x['procedimiento_ile']

    id_ile = [i for i in range(np.shape(mot)[0])]

    dic = {
        'id_ile': id_ile,
        'Motivos_interrupción' : mot,
        'Analgesico' : analg, 
        'Anticonceptivo_post' : anti,
        'Resultado_ile' : res,
        'Procediminiento_ile' : procd
    }

    pd.DataFrame(dic).to_csv(path, index = False)

In [21]:
def get_geo(x, path = './tablas/Geografia.csv', columna = 'alc_o_municipio',ile = True):
    """
    Función que crea la tabla Geografia para la base de datos csv
    x : Dataframe
    path : ruta donde se va a guardar el archivo csv
    """

    alcaldias_min = ['Alvaro Obregon', 'Azcapotzalco', 'Benito Juarez', 'Coyoacan', 'Cuajimalpa de Morelos', 'Cuauhtemoc', 'Gustavo A. Madero', 'Iztacalco','Iztapalapa', 'Magdalena Contreras', 'Miguel Hidalgo', 'Milpa Alta', 'Tlahuac', 'Talpan', 'Venustiano Carranza', 'Xochimilco']
    
    alcaldias = ['ÁLVARO OBREGÓN', 'AZCAPOTZALCO', 'BENÍTO JUÁREZ', 'COYOACÁN', 'CUAJIMALPA DE MORELOS', 'CUAUHTÉMOC', 'GUSTAVO A. MADERO', 'IZTACALCO','IZTAPALAPA', 'MAGDALENA CONTRERAS', 'MIGUEL HIDALGO', 'MILPA ALTA', 'TLÁHUAC', 'TLALPAN', 'VENUSTIANO CARRANZA', 'XOCHIMILCO']
    
    
    if ile:
        # municipios = x[columna]
        # lon = np.shape(municipios)
        
        municipios = x[columna]
        le = np.shape(municipios)
        estado = list()
        # lon = np.shape(municipios)

        pais = ['México' for i in range(le[0])]
        
        for municipio in municipios:
            if municipio in alcaldias:
                estado.append('Ciudad de México')
            else:
                estado.append('Edo. de México')
        
        id = ['ile' + str(i) for i in range(le[0])]
        est_nac = ['Null' for i in range(le[0])]
        mun_nac = ['Null' for i in range(le[0])]



        dat = {
            'Id_Geo' : id,
            'Pais' : pais,
            'Estado_nacimiento' : est_nac,
            'Mun_alc_nacimiento' : mun_nac,
            'Entidad_ILE' : estado,
            'Alc_o_mun_ILE' : municipios
            }

        pd.DataFrame(dat).to_csv(path, index=False)
    
    else:    
        print('primer paso')
        # Obtenemos la columan con la que vamos a trabajar
        nat_mun = x['mun_alc_nacimiento']
        inicio = 90042
        final = inicio + np.shape(nat_mun)[0]
        
        
        id = ['Nat' + str(i) for i in range(inicio, final)]
        pais = ['México' for i in nat_mun]
        estado = [float('nan') for mun in nat_mun]
        municipios = ['Null' for i in range(np.shape(nat_mun)[0])]
        
        est_nac = x['estado_nacimiento']
        
        
        # print('Natalidad',np.shape(nat_mun))
        # print('id', np.shape(id))
        # print('pais', np.shape(pais))
        # print('estado', np.shape(estado))
        # print('municipios', np.shape(municipios))
        # print('est_nac', np.shape(est_nac))
        nat_dic = {
            'Id_Geo' : id,
            'Pais' : pais,
            'Estado_nacimiento' : est_nac,
            'Mun_alc_nacimiento' : nat_mun,
            'Entidad_ILE' : estado,
            'Alc_o_mun_ILE' : municipios
        }
        
        # print(np.shape(pd.DataFrame(nat_dic)))
        # print(nat_dic)
        # print(np.shape(nat_dic))
        try:
            ile_geo = pd.read_csv(path)
            unido = ile_geo.append(pd.DataFrame(nat_dic))
            unido.to_csv('./tablas/geo_unida.csv', index=False)
        except:
            print('No se pudo completar')

In [17]:
data.columns

Index(['año', 'mes', 'fecha_ingreso', 'referida', 'estado_civil', 'edad',
       'institucion_derechohab', 'nivel_edu', 'ocupacion', 'religion',
       'parentesco', 'entidad', 'alc_o_municipio', 'edad_primera_menstruacion',
       'edad_inicio_vida_sexual', 'fecha_ultima_menstruacion',
       'semanas_embarazo', 'numero_hijos', 'numero_embarazos',
       'numero_abortos', 'numero_partos', 'numero_cesareas', 'numero_iles',
       'recibio_consejeria', 'uso_anticonceptivo', 'fecha_primera_valoracion',
       'numero_consultas', 'motivos_interrupcion', 'fecha_ingreso_hosp',
       'fecha_egreso_hosp', 'desc_servicio', 'semanas_gestacion_usg',
       'dias_gestacion_usg', 'firma_consentimiento', 'se_complica',
       'con_dolor', 'analgesico', 'anticonceptivo_post', 'fecha_cierre',
       'resultado_ile', 'procedimiento_ile'],
      dtype='object')

In [23]:
get_geo(data, ile=True)

## Dataset Natalidad


In [4]:
nat = pd.read_csv('./datasets/Natalidad_2016-2021.csv', encoding='utf-8')
nat.head(2)

,anio,mes,fecha_nacimiento,sexo,pais_nacimiento,estado_nacimiento,mun_alc_nacimiento,tutora_1_edad,tutora_1_sexo,tutora_1_nacionalidad,tutora_2_edad,tutora_2_sexo,tutora_2_nacionalidad
0,2016,Enero,2016-01-01,Mujer,MÃ©xico,Ciudad de MÃ©xico,Cuauhtemoc,32.0,Hombre,MÃ©xico,32.0,Mujer,MÃ©xico
1,2016,Enero,2016-01-01,Mujer,MÃ©xico,Ciudad de MÃ©xico,Venustiano Carranza,16.0,Hombre,MÃ©xico,15.0,Mujer,MÃ©xico


In [24]:
get_geo(nat, False)

KeyError: 'alc_o_municipio'

In [40]:
generar_tabla_tiempo(nat, columna='fecha_nacimiento')

C:\Users\rod_e\AppData\Local\Temp\ipykernel_16708\2857782416.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  juntos = uni.append(pd.DataFrame(tabla_tiempo))


In [25]:
get_geo(nat, columna='fecha_nacimiento', ile=False)

C:\Users\rod_e\AppData\Local\Temp\ipykernel_6516\215030511.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unido = ile_geo.append(pd.DataFrame(nat_dic))


In [5]:
generar_tabla_tiempo(nat, columna='fecha_nacimiento')

C:\Users\rod_e\AppData\Local\Temp\ipykernel_8552\89286029.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  juntos = uni.append(pd.DataFrame(tabla_tiempo))
